In [1]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import cmlreaders as cml
from ptsa.data.readers import EEGReader
from ptsa.data.MatlabIO import read_single_matlab_matrix_as_numpy_structured_array

### Grab raw events and montage

One very helpful function for reading matlab events!!

In [2]:
events = pd.DataFrame.from_records(
    read_single_matlab_matrix_as_numpy_structured_array(
        '/data/eeg/UT159/behavioral/FR1_scopolamine/session_1/events.mat', 'events'
    )
)
events # note that the eegfile is wrong

,subject,session,list,serialpos,type,item,itemno,recalled,mstime,msoffset,...,intrusion,isStim,expVersion,stimLoc,stimAmp,stimAnode,stimCathode,stimList,eegfile,eegoffset
0,UT159,1,-999,-999,B,X,-999,-999,1.564763e+12,0,...,-999,-999,,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,109250
1,UT159,1,-999,-999,SESS_START,X,-999,-999,1.564763e+12,0,...,-999,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,109364
2,UT159,1,-999,-999,COUNTDOWN_START,X,-999,-999,1.564763e+12,0,...,-999,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,131034
3,UT159,1,-999,-999,COUNTDOWN_END,X,-999,-999,1.564763e+12,0,...,-999,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,141461
4,UT159,1,-999,-999,PRACTICE_WORD,X,-999,-999,1.564763e+12,1,...,-999,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,145136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,UT159,1,25,-999,REC_WORD,MOLE,155,-999,1.564766e+12,-999,...,0,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,2661095
692,UT159,1,25,-999,REC_WORD,HOLE,124,-999,1.564766e+12,-999,...,0,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,2664950
693,UT159,1,25,-999,REC_END,X,-999,-999,1.564766e+12,1,...,-999,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,2686604
694,UT159,1,25,-999,SESS_END,X,-999,-999,1.564766e+12,0,...,-999,-999,v_1.05,X,NaN,NaN,NaN,-999,/project/TIBIR/Lega_lab/shared/lega_ansir/subj...,2696252


In [4]:
contacts = pd.DataFrame.from_records(np.recfromtxt("/data/eeg/UT159/eeg.noreref/jacksheet.txt", encoding='utf-8'))
contacts = contacts.rename(columns={'f0':'contact', 'f1':'label'})
contacts[:10]

,contact,label
0,1,RB1
1,2,RB2
2,3,RB3
3,4,RB4
4,5,RB5
5,6,RB6
6,7,RB7
7,8,RB8
8,9,RB9
9,10,RB10


### Localization info
Only one thing left: grab the localization info. Looks like it's not so consistently formatted (would be better as a .csv) so maybe necessary to parse line by line

In [5]:
with open("/data/eeg/UT159/docs/UT159_depth_el_info.txt", 'r') as f:
    lines = f.readlines()
lines = [line.rstrip('\n') for line in lines]

In [6]:
depth_info = pd.DataFrame(columns=['label', 'description'])
for line in lines:
    split = line.split()
    try:
        depth_info = depth_info.append({'label':split[1], 'description':' '.join(split[2:])}, ignore_index=True)
    except IndexError:
        continue
contacts = depth_info.merge(contacts, how='inner')    
contacts[:10]

,label,description,contact
0,RB1,Anterior hippocampus,1
1,RB2,Anterior hippocampus/WM,2
2,RB3,WM,3
3,RB4,WM,4
4,RB5,WM,5
5,RB6,WM,6
6,RB7,STS/STG,7
7,RB8,STS/STG,8
8,RB9,STS/STG,9
9,RB10,OUT,10


Now we have an updated contacts df with anatomic locations from the UTSW localizations!

## Read EEG
Next, use the base file name in eeg.noreref to read the binary channel data

In [6]:
# We want to get the base file name in a clean/systematic way, probably by globbing on the experiment_session name 
from glob import glob
import os
root = '/data/eeg'
subject = 'UT159'
experiment = 'FR1_scopolamine'
session = 1
paths = glob(os.path.join(root, subject, 'eeg.noreref', subject+'_'+experiment+'_'+str(session)+'*'), recursive=True)
file_base = os.path.basename(paths[0]).split('.')[0]
base_name = os.path.join(os.path.dirname(paths[0]), file_base)
base_name

'/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolamine_1_02Aug19_1109'

In [8]:
events['eegfile'] = base_name
events = events[events.type=='WORD']
events[0:10]

,subject,session,list,serialpos,type,item,itemno,recalled,mstime,msoffset,...,intrusion,isStim,expVersion,stimLoc,stimAmp,stimAnode,stimCathode,stimList,eegfile,eegoffset
24,UT159,1,1,1,WORD,STOVE,256,0,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,261401
25,UT159,1,1,2,WORD,CALF,41,1,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,263802
26,UT159,1,1,3,WORD,MUG,162,0,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,266185
27,UT159,1,1,4,WORD,APE,2,1,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,268802
28,UT159,1,1,5,WORD,FOX,104,1,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,271219
29,UT159,1,1,6,WORD,WOLF,295,1,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,273836
30,UT159,1,1,7,WORD,HOOK,126,1,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,276436
31,UT159,1,1,8,WORD,BARN,11,0,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,278970
32,UT159,1,1,9,WORD,NOSE,167,0,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,281421
33,UT159,1,1,10,WORD,HEN,122,1,1.564763e+12,1,...,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,283838


In [9]:
eeg = EEGReader(events.to_records(),
                channels= contacts.contact.values[contacts.description.str.contains('hippocampus')],
                start_time=0, end_time=0.3).read()

In [10]:
eeg

<xarray.TimeSeries (channels: 8, events: 300, time: 300)>
array([[[-1.36718750e+00,  1.01562500e+01,  2.08984375e+01, ...,
          9.08203125e+01,  7.48046875e+01,  6.05468750e+01],
        [-1.19140625e+02, -1.35546875e+02, -1.45410156e+02, ...,
         -2.26367188e+02, -2.12011719e+02, -2.03906250e+02],
        [-1.23925781e+02, -1.24609375e+02, -1.22656250e+02, ...,
          7.58789062e+01,  7.99804688e+01,  8.31054688e+01],
        ...,
        [-2.55859375e+01, -1.37695312e+01, -2.73437500e+00, ...,
          3.99414062e+01,  4.73632812e+01,  5.39062500e+01],
        [ 2.58203125e+02,  2.62109375e+02,  2.64941406e+02, ...,
          3.66992188e+02,  3.74707031e+02,  3.77929688e+02],
        [ 9.00390625e+01,  1.04589844e+02,  1.17578125e+02, ...,
          2.00000000e+02,  2.13671875e+02,  2.20117188e+02]],

       [[-1.84667969e+02, -1.82226562e+02, -1.79785156e+02, ...,
         -2.33886719e+02, -2.33105469e+02, -2.30273438e+02],
        [-9.84375000e+01, -1.00097656e+02, -1

In [11]:
from ptsa.data.filters import MorletWaveletFilter

In [12]:
MorletWaveletFilter(eeg, [3., 8.], output='power').filter()

CPP total time wavelet loop:  0.19980406761169434


<xarray.TimeSeries (frequency: 2, channels: 8, events: 300, time: 300)>
array([[[[2.53483184e+08, 2.55638329e+08, 2.57808635e+08, ...,
          1.01598157e+09, 1.01602055e+09, 1.01602668e+09],
         [5.77712426e+08, 5.80784214e+08, 5.83858168e+08, ...,
          9.58642168e+08, 9.56779402e+08, 9.54896535e+08],
         [9.77634447e+08, 9.82037241e+08, 9.86435611e+08, ...,
          1.22149397e+09, 1.21770368e+09, 1.21389316e+09],
         ...,
         [1.24222412e+08, 1.24921435e+08, 1.25621151e+08, ...,
          2.07801672e+08, 2.07340843e+08, 2.06875495e+08],
         [1.12598845e+08, 1.12577344e+08, 1.12560551e+08, ...,
          4.55024130e+08, 4.56216803e+08, 4.57394660e+08],
         [1.65602529e+08, 1.66649031e+08, 1.67698101e+08, ...,
          3.46200820e+08, 3.45680103e+08, 3.45150754e+08]],

        [[3.99522104e+08, 3.99862547e+08, 4.00200401e+08, ...,
          5.51297318e+08, 5.51419055e+08, 5.51528879e+08],
         [1.65134404e+07, 1.64568046e+07, 1.63998094e+07, 